In [10]:
# 218. The Skyline Problem
class Solution(object):
    def getSkyline(self, blds):
        """
        :type blds: List[List[int]]
        :rtype: List[List[int]]
        """
        # blds: [Li, Ri, Hi]
        if not blds:
            return []
        if len(blds) == 1:
            return [[blds[0][0],blds[0][2]], [blds[0][1],0]]
        mid = len(blds) // 2
        left = self.getSkyline(blds[:mid])
        right = self.getSkyline(blds[mid:])
        return self.merge(left, right)
    
    def merge(self, left, right):
        maxH = leftH = rightH = 0
        pl = pr = 0
        nl = len(left)
        nr = len(right)
        res = []
        while pl < nl and pr < nr:
            if left[pl][0] < right[pr][0]:
                pos, leftH = left[pl] # returns bottom left
                pl += 1
            elif left[pl][0] > right[pr][0]:
                pos, rightH = right[pr]
                pr += 1
            else:
                pos, leftH = left[pl]
                rightH = right[pr][1]
                pl += 1
                pr += 1
            maxH = max(leftH, rightH)
            # update result
            if not res or maxH != res[-1][1]:
                res += [[pos, maxH]]
        if pl < nl:
            res += left[pl:]
        if pr < nr:
            res += right[pr:]
        return res
        
blds = [[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]]
Solution().getSkyline(blds)

2 3
9 3
9 7
15 19
20 19
20 24
5 15
12 15
2 5
3 5
7 5
7 12
9 12


[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [11]:
# 218. The Skyline Problem
class Solution(object):
    def getSkyline(self, blds):
        """
        :type blds: List[List[int]]
        :rtype: List[List[int]]
        """
        # blds: [Li, Ri, Hi]
        if not blds: return []
        if len(blds) == 1: return [[blds[0][0], blds[0][2]], [blds[0][1], 0]]
        mid = len(blds) // 2
        left = self.getSkyline(blds[:mid])
        right = self.getSkyline(blds[mid:])
        return self.merge(left, right)
    
    def merge(self, left, right):
        h1, h2, res = 0, 0, []
        while left and right:
            if left[0][0] < right[0][0]:
                pos, h1 = left[0]
                left = left[1:]
            elif left[0][0] > right[0][0]:
                pos, h2 = right[0]
                right = right[1:]
            else:
                pos, h1 = left[0]
                h2 = right[0][1]
                left = left[1:]
                right = right[1:]
            H = max(h1, h2)
            if not res or H != res[-1][1]:
                res.append([pos, H])
        if left:
            res += left
        if right:
            res += right
        return res
    
# buildings = [[2,9,10],[3,7,15]]
buildings = [[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]]
Solution().getSkyline(buildings)

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]